In [2]:
import os, sys, time, gc, warnings, math, subprocess, importlib
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

warnings.filterwarnings('ignore')
np.set_printoptions(suppress=True)
pd.set_option('display.max_columns', 200)

SEED = 42
N_SPLITS = 5
RANDOM_STATE = 42

def set_seed(seed: int = 42):
    import random
    random.seed(seed)
    np.random.seed(seed)

def ensure_package(pkg_name: str, import_name: str = None, extra_install: str = ''):
    name = import_name or pkg_name
    try:
        return importlib.import_module(name)
    except ImportError:
        print(f"[INFO] Installing {pkg_name}...")
        cmd = [sys.executable, '-m', 'pip', 'install', pkg_name] + ([extra_install] if extra_install else [])
        subprocess.check_call(cmd)
        return importlib.import_module(name)

set_seed(SEED)

# Ensure catboost
catboost = ensure_package('catboost', 'catboost')
from catboost import CatBoostClassifier, Pool

t0 = time.time()
print('[INFO] Loading data...')
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
print(f"[INFO] train shape: {train.shape}, test shape: {test.shape}")
assert 'Cover_Type' in train.columns

# Target and base features
y = train['Cover_Type'].values - 1
feature_cols = [c for c in train.columns if c not in ['Cover_Type', 'Id']]
X = train[feature_cols].copy()
X_test = test[[c for c in test.columns if c != 'Id']].copy()

# --- Feature Engineering to match main.ipynb ---
print('[INFO] Feature engineering...')
elev_threshold = X_test['Elevation'].median()
X['is_high_elevation'] = (X['Elevation'] > elev_threshold).astype(np.int8)
X_test['is_high_elevation'] = (X_test['Elevation'] > elev_threshold).astype(np.int8)

if set(['Horizontal_Distance_To_Hydrology','Vertical_Distance_To_Hydrology']).issubset(X.columns):
    X['Hydrology_Euclid'] = np.sqrt(X['Horizontal_Distance_To_Hydrology']**2 + X['Vertical_Distance_To_Hydrology']**2)
    X_test['Hydrology_Euclid'] = np.sqrt(X_test['Horizontal_Distance_To_Hydrology']**2 + X_test['Vertical_Distance_To_Hydrology']**2)
    X['Elev_minus_VertHydro'] = X['Elevation'] - X['Vertical_Distance_To_Hydrology']
    X_test['Elev_minus_VertHydro'] = X_test['Elevation'] - X_test['Vertical_Distance_To_Hydrology']

hill_cols = [c for c in ['Hillshade_9am','Hillshade_Noon','Hillshade_3pm'] if c in X.columns]
if len(hill_cols) == 3:
    X['Hillshade_Mean'] = X[hill_cols].mean(axis=1)
    X_test['Hillshade_Mean'] = X_test[hill_cols].mean(axis=1)
    X['Hillshade_Min'] = X[hill_cols].min(axis=1)
    X_test['Hillshade_Min'] = X_test[hill_cols].min(axis=1)
    X['Hillshade_Max'] = X[hill_cols].max(axis=1)
    X_test['Hillshade_Max'] = X_test[hill_cols].max(axis=1)
    X['Hillshade_Range'] = X['Hillshade_Max'] - X['Hillshade_Min']
    X_test['Hillshade_Range'] = X_test['Hillshade_Max'] - X_test['Hillshade_Min']

dist_cols = ['Horizontal_Distance_To_Fire_Points','Horizontal_Distance_To_Roadways','Horizontal_Distance_To_Hydrology']
if set(dist_cols).issubset(X.columns):
    hf, rr, hh = dist_cols
    X['DistDiff_Fire_Road'] = (X[hf] - X[rr]).abs()
    X_test['DistDiff_Fire_Road'] = (X_test[hf] - X_test[rr]).abs()
    X['DistDiff_Fire_Hydro'] = (X[hf] - X[hh]).abs()
    X_test['DistDiff_Fire_Hydro'] = (X_test[hf] - X_test[hh]).abs()
    X['DistDiff_Road_Hydro'] = (X[rr] - X[hh]).abs()
    X_test['DistDiff_Road_Hydro'] = (X_test[rr] - X_test[hh]).abs()
    X['DistMean_FRH'] = (X[hf] + X[rr] + X[hh]) / 3.0
    X_test['DistMean_FRH'] = (X_test[hf] + X_test[rr] + X_test[hh]) / 3.0
    X['DistSum_FRH'] = (X[hf] + X[rr] + X[hh])
    X_test['DistSum_FRH'] = (X_test[hf] + X_test[rr] + X_test[hh])
    X['DistMin_FRH'] = X[[hf, rr, hh]].min(axis=1)
    X_test['DistMin_FRH'] = X_test[[hf, rr, hh]].min(axis=1)
    X['DistMax_FRH'] = X[[hf, rr, hh]].max(axis=1)
    X_test['DistMax_FRH'] = X_test[[hf, rr, hh]].max(axis=1)

soil_cols = [c for c in X.columns if c.startswith('Soil_Type_')]
wild_cols = [c for c in X.columns if c.startswith('Wilderness_Area_')]
if soil_cols:
    X['Soil_Type_Count'] = X[soil_cols].sum(axis=1)
    X_test['Soil_Type_Count'] = X_test[soil_cols].sum(axis=1)
if wild_cols:
    X['Wilderness_Area_Count'] = X[wild_cols].sum(axis=1)
    X_test['Wilderness_Area_Count'] = X_test[wild_cols].sum(axis=1)

if 'Aspect' in X.columns:
    X['Aspect_sin'] = np.sin(np.deg2rad(X['Aspect']))
    X_test['Aspect_sin'] = np.sin(np.deg2rad(X_test['Aspect']))
    X['Aspect_cos'] = np.cos(np.deg2rad(X['Aspect']))
    X_test['Aspect_cos'] = np.cos(np.deg2rad(X_test['Aspect']))

features = X.columns.tolist()
print(f"[INFO] Final feature count: {len(features)}")

# Save/load consistent folds for cross-model reproducibility
fold_file = 'fold_indices.npy'
if os.path.exists(fold_file):
    folds = np.load(fold_file, allow_pickle=True).tolist()
else:
    skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_STATE)
    folds = list(skf.split(X, y))
    np.save(fold_file, np.array(folds, dtype=object))
    print(f"[INFO] Saved fold indices to {fold_file}")

# CatBoost params: try GPU if available, else CPU
use_gpu = False
try:
    from catboost.utils import get_gpu_device_count
    use_gpu = get_gpu_device_count() > 0
except Exception:
    use_gpu = False
print(f"[INFO] CatBoost use_gpu={use_gpu}")

params = dict(
    loss_function='MultiClass',
    eval_metric='Accuracy',
    iterations=5000,
    learning_rate=0.04,
    depth=8,
    l2_leaf_reg=3.0,
    random_seed=SEED,
    random_strength=1.0,
    bagging_temperature=1.0,
    od_wait=200,
    od_type='Iter',
    verbose=100,
    task_type='GPU' if use_gpu else 'CPU',
    devices='0' if use_gpu else None,
    thread_count=36
)

oof_preds = np.zeros((X.shape[0], 7), dtype=np.float32)
test_preds = np.zeros((X_test.shape[0], 7), dtype=np.float32)
fold_acc = []

print('[INFO] Starting CatBoost 5-fold CV...')
for fold, (trn_idx, val_idx) in enumerate(folds, 1):
    f_t = time.time()
    print(f"[FOLD {fold}/{N_SPLITS}] Train: {len(trn_idx)}, Valid: {len(val_idx)}")
    X_trn = X.iloc[trn_idx]
    y_trn = y[trn_idx]
    X_val = X.iloc[val_idx]
    y_val = y[val_idx]

    train_pool = Pool(X_trn, y_trn)
    valid_pool = Pool(X_val, y_val)

    model = CatBoostClassifier(**params)
    model.fit(train_pool, eval_set=valid_pool, use_best_model=True)

    val_pred_proba = model.predict_proba(valid_pool)
    oof_preds[val_idx] = val_pred_proba
    val_pred = np.argmax(val_pred_proba, axis=1)
    acc = accuracy_score(y_val, val_pred)
    fold_acc.append(acc)
    print(f"[FOLD {fold}] ACC={acc:.6f} | best_iter={model.get_best_iteration()} | elapsed={time.time()-f_t:.1f}s")

    test_fold_pred = model.predict_proba(X_test)
    test_preds += test_fold_pred / N_SPLITS

    del X_trn, X_val, y_trn, y_val, train_pool, valid_pool, model, val_pred_proba, test_fold_pred
    gc.collect()

oof_pred_labels = np.argmax(oof_preds, axis=1)
cv_acc = accuracy_score(y, oof_pred_labels)
print(f"[CV] CatBoost Mean ACC: {np.mean(fold_acc):.6f}; OOF ACC: {cv_acc:.6f}")

# Save preds for ensembling
np.save('cat_oof_preds.npy', oof_preds)
np.save('cat_test_preds.npy', test_preds)
print('[INFO] Saved cat_oof_preds.npy and cat_test_preds.npy')

# Optional quick submission from CatBoost alone
sub = pd.DataFrame({
    'Id': test['Id'].values if 'Id' in test.columns else np.arange(len(test)),
    'Cover_Type': np.argmax(test_preds, axis=1) + 1
})
sub.to_csv('submission_catboost.csv', index=False)
print('[INFO] Saved submission_catboost.csv')
print(f"[DONE] Total elapsed: {time.time()-t0:.1f}s")